
## **🚨 Problema y Violación del Principio de Pure Fabrication**
- **Pure Fabrication** establece que ciertas responsabilidades **no deben asignarse a entidades del dominio** si no están directamente relacionadas con su propósito.  
- En este caso, la clase `UserProfile` **manipula imágenes de perfil de los usuarios**, lo cual **no es su responsabilidad**.  
- **El código mezcla lógica de negocio con procesamiento de imágenes**, haciendo que `UserProfile` sea difícil de probar, extender y mantener.  
- **No existe una capa separada** para manejar la lógica de imágenes, lo que hace que `UserProfile` dependa directamente de bibliotecas como `PIL` (Pillow).

---

## **🚨 Código que Viola Pure Fabrication**
```python
from PIL import Image, ImageFilter
from typing import Optional

class UserProfile:
    """Clase que maneja el perfil del usuario y también la manipulación de su imagen de perfil."""
    def __init__(self, username: str, image_path: str) -> None:
        self.username = username
        self.image_path = image_path

    def apply_grayscale_filter(self) -> None:
        """Aplica un filtro de escala de grises a la imagen del usuario."""
        image = Image.open(self.image_path).convert("L")
        image.save(self.image_path)
        print(f"Se aplicó escala de grises a la imagen de {self.username}")

    def resize_profile_picture(self, width: int, height: int) -> None:
        """Redimensiona la imagen de perfil del usuario."""
        image = Image.open(self.image_path)
        resized_image = image.resize((width, height))
        resized_image.save(self.image_path)
        print(f"Imagen de {self.username} redimensionada a {width}x{height}")

    def blur_profile_picture(self) -> None:
        """Aplica un filtro de desenfoque a la imagen del usuario."""
        image = Image.open(self.image_path)
        blurred_image = image.filter(ImageFilter.BLUR)
        blurred_image.save(self.image_path)
        print(f"Imagen de {self.username} desenfocada")

# Uso del código violando Pure Fabrication
user = UserProfile("john_doe", "profile.jpg")
user.apply_grayscale_filter()
user.resize_profile_picture(128, 128)
user.blur_profile_picture()
```

---

## **🔴 ¿Por qué este código viola Pure Fabrication?**
1. **`UserProfile` mezcla lógica de negocio con procesamiento de imágenes.**  
   - `UserProfile` debería manejar datos del usuario, **no la manipulación de imágenes**.  
   - Si la biblioteca `PIL` cambia, `UserProfile` se verá afectado innecesariamente.

2. **Dificulta la escalabilidad y pruebas unitarias.**  
   - Si en el futuro queremos cambiar la forma de procesar imágenes (por ejemplo, usando OpenCV en lugar de PIL), tendríamos que modificar `UserProfile`.

3. **Falta de modularidad y reutilización.**  
   - La lógica de manipulación de imágenes no se puede reutilizar en otras partes del sistema sin depender de `UserProfile`.

---

## **✅ Solución: Crear una Clase de Fabricación Separada**
Para corregir el problema, aplicamos **Pure Fabrication**, creando una clase separada **`ImageProcessor`** que maneja el procesamiento de imágenes.  
Esto **desacopla `UserProfile` del manejo de imágenes**, facilitando el mantenimiento y la escalabilidad.

---

### **✅ Código Mejorado**
```python
from PIL import Image, ImageFilter
from typing import Optional

class ImageProcessor:
    """Clase independiente encargada de manipular imágenes (Pure Fabrication)."""
    @staticmethod
    def apply_grayscale(image_path: str) -> None:
        """Aplica escala de grises a una imagen."""
        image = Image.open(image_path).convert("L")
        image.save(image_path)
        print(f"✅ Imagen {image_path} convertida a escala de grises.")

    @staticmethod
    def resize(image_path: str, width: int, height: int) -> None:
        """Redimensiona una imagen."""
        image = Image.open(image_path)
        resized_image = image.resize((width, height))
        resized_image.save(image_path)
        print(f"✅ Imagen {image_path} redimensionada a {width}x{height}.")

    @staticmethod
    def apply_blur(image_path: str) -> None:
        """Aplica un filtro de desenfoque a una imagen."""
        image = Image.open(image_path)
        blurred_image = image.filter(ImageFilter.BLUR)
        blurred_image.save(image_path)
        print(f"✅ Imagen {image_path} desenfocada.")

class UserProfile:
    """Clase que representa el perfil de usuario sin manejar imágenes directamente."""
    def __init__(self, username: str, image_path: str) -> None:
        self.username = username
        self.image_path = image_path

# Uso del código refactorizado con Pure Fabrication
user = UserProfile("john_doe", "profile.jpg")

# Ahora la manipulación de imágenes está separada
ImageProcessor.apply_grayscale(user.image_path)
ImageProcessor.resize(user.image_path, 128, 128)
ImageProcessor.apply_blur(user.image_path)
```

---

## **✅ ¿Por qué esta versión es mejor?**
✅ **Separa completamente la lógica de negocio de la manipulación de imágenes.**  
   - `UserProfile` **ya no sabe nada** sobre cómo se procesan las imágenes.  
   - `ImageProcessor` maneja toda la lógica de manipulación de imágenes.  

✅ **Hace el código más reutilizable y escalable.**  
   - `ImageProcessor` se puede usar en cualquier parte del sistema sin depender de `UserProfile`.  
   - Si agregamos una funcionalidad de **watermark (marca de agua)**, simplemente la añadimos a `ImageProcessor`, sin tocar `UserProfile`.  

✅ **Facilita la prueba de cada componente.**  
   - Podemos hacer pruebas unitarias en `ImageProcessor` sin necesidad de crear instancias de `UserProfile`.  

✅ **Cumple Open/Closed Principle (OCP).**  
   - Si queremos usar OpenCV en lugar de PIL, solo modificamos `ImageProcessor`, sin afectar `UserProfile`.  

---

## **🚀 Comparación Antes vs. Después**
| Problema | Código Anterior 🚨 | Código Mejorado ✅ |
|----------|-------------------|-------------------|
| **Acoplamiento** | `UserProfile` estaba directamente acoplado a `PIL`. | `UserProfile` no sabe nada sobre imágenes. |
| **Reutilización** | No se podía usar la lógica de imagen en otros contextos. | `ImageProcessor` es independiente y reutilizable. |
| **Mantenibilidad** | Cualquier cambio en `PIL` afectaría `UserProfile`. | `UserProfile` es inmune a cambios en el procesamiento de imágenes. |
| **Extensibilidad** | Agregar nuevas funciones de imagen requería modificar `UserProfile`. | Podemos agregar funciones en `ImageProcessor` sin tocar `UserProfile`. |

---

## **🎯 Conclusión**
El código original **violaba Pure Fabrication** porque `UserProfile` tenía responsabilidades que no le correspondían.  
La versión refactorizada **corrige este problema** al crear una clase independiente `ImageProcessor`, que encapsula toda la lógica de procesamiento de imágenes.


In [ ]:
from PIL import Image, ImageFilter
from typing import Optional

class ImageProcessor:
    """Clase independiente encargada de manipular imágenes (Pure Fabrication)."""
    @staticmethod
    def apply_grayscale(image_path: str) -> None:
        """Aplica escala de grises a una imagen."""
        image = Image.open(image_path).convert("L")
        image.save(image_path)
        print(f"✅ Imagen {image_path} convertida a escala de grises.")

    @staticmethod
    def resize(image_path: str, width: int, height: int) -> None:
        """Redimensiona una imagen."""
        image = Image.open(image_path)
        resized_image = image.resize((width, height))
        resized_image.save(image_path)
        print(f"✅ Imagen {image_path} redimensionada a {width}x{height}.")

    @staticmethod
    def apply_blur(image_path: str) -> None:
        """Aplica un filtro de desenfoque a una imagen."""
        image = Image.open(image_path)
        blurred_image = image.filter(ImageFilter.BLUR)
        blurred_image.save(image_path)
        print(f"✅ Imagen {image_path} desenfocada.")

class UserProfile:
    """Clase que representa el perfil de usuario sin manejar imágenes directamente."""
    def __init__(self, username: str, image_path: str) -> None:
        self.username = username
        self.image_path = image_path

# Uso del código refactorizado con Pure Fabrication
user = UserProfile("john_doe", "profile.jpg")

# Ahora la manipulación de imágenes está separada
ImageProcessor.apply_grayscale(user.image_path)
ImageProcessor.resize(user.image_path, 128, 128)
ImageProcessor.apply_blur(user.image_path)


# Hay lugar para mejoras?

Sí, aunque la refactorización ya **respeta el principio de Pure Fabrication**, todavía **hay margen de mejora** desde el punto de vista de diseño y arquitectura. Vamos a analizar las áreas que aún pueden optimizarse y refactorizar el código para hacerlo aún más robusto y flexible.

---

## **🔍 Problemas que persisten en la versión refactorizada**
1. **`ImageProcessor` sigue dependiendo de `PIL` (Pillow) directamente en los métodos estáticos.**  
   - Aunque ya desacoplamos `UserProfile`, `ImageProcessor` está **acoplado a `PIL`**, lo que hace difícil cambiar la implementación si queremos usar otra librería, como OpenCV (`cv2`).  

2. **No hay una interfaz para abstracción de procesamiento de imágenes.**  
   - Si en el futuro queremos soportar múltiples estrategias de procesamiento (PIL, OpenCV, etc.), tendríamos que modificar `ImageProcessor`, lo que rompe **Open/Closed Principle (OCP)**.  

3. **La clase `ImageProcessor` es estática, lo que limita su flexibilidad.**  
   - Usar solo métodos estáticos puede **dificultar pruebas unitarias y extensibilidad**.  

4. **No se maneja la posibilidad de errores en la manipulación de imágenes.**  
   - Si la imagen no existe o hay un error de lectura, el código **fallará en tiempo de ejecución** sin un manejo adecuado de excepciones.  

---

## **🚀 Solución: Implementar una Interfaz de Procesamiento de Imágenes y Soporte para Múltiples Backends**
Para mejorar el diseño:
✅ **Creamos una interfaz `ImageProcessorInterface`** para desacoplar la implementación de `PIL`.  
✅ **Permitimos múltiples estrategias de procesamiento** (`PILImageProcessor`, `OpenCVImageProcessor`).  
✅ **Convertimos `ImageProcessor` en una fábrica** que selecciona la implementación correcta en tiempo de ejecución.  
✅ **Agregamos manejo de excepciones para mejorar la robustez**.  

---

## **✅ Código Mejorado**
```python
from typing import Protocol, Optional
from PIL import Image, ImageFilter
import cv2
import os

class ImageProcessorInterface(Protocol):
    """Interfaz que define las operaciones de procesamiento de imágenes."""
    def apply_grayscale(self, image_path: str) -> None:
        ...

    def resize(self, image_path: str, width: int, height: int) -> None:
        ...

    def apply_blur(self, image_path: str) -> None:
        ...

class PILImageProcessor(ImageProcessorInterface):
    """Implementación de procesamiento de imágenes usando PIL (Pillow)."""
    def apply_grayscale(self, image_path: str) -> None:
        try:
            image = Image.open(image_path).convert("L")
            image.save(image_path)
            print(f"✅ Imagen {image_path} convertida a escala de grises usando PIL.")
        except Exception as e:
            print(f"❌ Error al aplicar escala de grises con PIL: {e}")

    def resize(self, image_path: str, width: int, height: int) -> None:
        try:
            image = Image.open(image_path)
            resized_image = image.resize((width, height))
            resized_image.save(image_path)
            print(f"✅ Imagen {image_path} redimensionada a {width}x{height} usando PIL.")
        except Exception as e:
            print(f"❌ Error al redimensionar imagen con PIL: {e}")

    def apply_blur(self, image_path: str) -> None:
        try:
            image = Image.open(image_path)
            blurred_image = image.filter(ImageFilter.BLUR)
            blurred_image.save(image_path)
            print(f"✅ Imagen {image_path} desenfocada usando PIL.")
        except Exception as e:
            print(f"❌ Error al aplicar desenfoque con PIL: {e}")

class OpenCVImageProcessor(ImageProcessorInterface):
    """Implementación de procesamiento de imágenes usando OpenCV."""
    def apply_grayscale(self, image_path: str) -> None:
        try:
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                raise ValueError("No se pudo leer la imagen.")
            cv2.imwrite(image_path, image)
            print(f"✅ Imagen {image_path} convertida a escala de grises usando OpenCV.")
        except Exception as e:
            print(f"❌ Error al aplicar escala de grises con OpenCV: {e}")

    def resize(self, image_path: str, width: int, height: int) -> None:
        try:
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError("No se pudo leer la imagen.")
            resized_image = cv2.resize(image, (width, height))
            cv2.imwrite(image_path, resized_image)
            print(f"✅ Imagen {image_path} redimensionada a {width}x{height} usando OpenCV.")
        except Exception as e:
            print(f"❌ Error al redimensionar imagen con OpenCV: {e}")

    def apply_blur(self, image_path: str) -> None:
        try:
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError("No se pudo leer la imagen.")
            blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
            cv2.imwrite(image_path, blurred_image)
            print(f"✅ Imagen {image_path} desenfocada usando OpenCV.")
        except Exception as e:
            print(f"❌ Error al aplicar desenfoque con OpenCV: {e}")

class ImageProcessorFactory:
    """Fábrica para seleccionar el backend de procesamiento de imágenes."""
    @staticmethod
    def get_processor(use_opencv: bool = False) -> ImageProcessorInterface:
        if use_opencv:
            return OpenCVImageProcessor()
        return PILImageProcessor()

class UserProfile:
    """Modelo de usuario sin lógica de imágenes."""
    def __init__(self, username: str, image_path: str) -> None:
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"❌ La imagen '{image_path}' no existe.")
        self.username = username
        self.image_path = image_path

# Uso del código mejorado con fábrica y soporte para OpenCV o PIL
use_opencv = False  # Cambiar a True para usar OpenCV
processor = ImageProcessorFactory.get_processor(use_opencv)

user = UserProfile("john_doe", "profile.jpg")
processor.apply_grayscale(user.image_path)
processor.resize(user.image_path, 128, 128)
processor.apply_blur(user.image_path)
```

---

## **🎯 ¿Qué mejoras se lograron?**
✅ **Se desacopla completamente `ImageProcessor` de una implementación específica**  
   - Ahora podemos usar **PIL o OpenCV sin modificar el código principal**.  
   - Si en el futuro se quiere usar `TensorFlow`, `ImageMagick` o cualquier otro backend, solo hay que implementar una nueva clase que herede de `ImageProcessorInterface`.  

✅ **Se mejora la extensibilidad sin modificar código existente (OCP - Open/Closed Principle)**  
   - Si queremos agregar un nuevo procesador de imágenes (`TensorFlowImageProcessor`), solo lo implementamos sin tocar `UserProfile`.  

✅ **Se introduce un patrón Factory para la selección del backend**  
   - `ImageProcessorFactory` permite cambiar entre PIL y OpenCV de forma **transparente**.  
   - Esto hace que el código sea más **configurable** y adaptable.  

✅ **Se añade manejo de errores y validación de archivos**  
   - Se validan excepciones en `PILImageProcessor` y `OpenCVImageProcessor`.  
   - `UserProfile` ahora **valida si la imagen existe** antes de instanciarse.  

---

## **📌 Comparación Antes vs. Después**
| Problema | Código Inicial 🚨 | Código Mejorado ✅ |
|----------|-------------------|-------------------|
| **Acoplamiento** | `ImageProcessor` dependía de `PIL`. | `ImageProcessor` usa una interfaz y una fábrica para desacoplarse de la implementación. |
| **Extensibilidad** | No se podía cambiar fácilmente la librería de imágenes. | Podemos cambiar entre PIL y OpenCV sin modificar el código base. |
| **Manejo de errores** | No validaba archivos ni excepciones. | Ahora maneja errores y valida si la imagen existe. |

---

## **🏆 Conclusión**
Esta versión **aplica completamente el principio de Pure Fabrication**, separando la lógica de imágenes en una **clase abstracta extensible**, permitiendo usar diferentes backends sin modificar el código principal. 🚀